# 「Python による経済・経営分析のための実践的データサイエンス」第3章 (pp.81-83)

パッケージのインストール

In [1]:
!pip install sparqlwrapper

Wikipedia にあるコンピューターゲームメーカーの情報をまとめて取得する

In [2]:
#提供されたサンプルのままだと、以下のエラーが起きる。これは、SSL証明書が正しくないサイトとのこと
#URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1131)>
#SSL証明書が正しくないサイトに対してPythonでアクセスする方法を追加する。
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from SPARQLWrapper import SPARQLWrapper

#コンピューターゲームメーカーの情報を取得する
sparql2 = SPARQLWrapper(endpoint='http://ja.dbpedia.org/sparql', returnFormat='json')
sparql2.setQuery("""                                                                                                                                                                      
    PREFIX dbpedia-owl:  <http://dbpedia.org/ontology/>                                                                                                                                  
    select distinct ?name ?abstract where {
    ?company <http://dbpedia.org/ontology/wikiPageWikiLink> <http://ja.dbpedia.org/resource/Category:日本のコンピュータゲームメーカー・ブランド> .
    ?company rdfs:label ?name .
    ?company <http://dbpedia.org/ontology/abstract> ?abstract .
}                                                                                                                   
""")
results2 = sparql2.query().convert()

結果を整形する

In [3]:
#結果を整形しPrint する
for result in results2["results"]["bindings"]:
    print(result["name"]["value"],";",result["abstract"]["value"])

オフィスエラン ; 株式会社オフィスエランは日本の企業。主にコンテンツに関する企画会社兼ソフトウェアメーカー。所在地は東京都世田谷区。 主にマルチメディア電子書籍の企画開発を行っており、同社では自社ノベルソフトを「オーディオビジュアルノベル」と呼称し、登録商標としている。 東映アニメーションによる「ゲゲゲの鬼太郎」プロモーションwebサイト企画から業務を開始し、その後ゲームコンテンツの企画・ムービー制作・音声収録と多岐にわたる業務を行っている。実写によるDVD制作も行う。 自社ソフトウェアブランドとして「トリニトライ」を運用しており、「ライフウィズアイドル」シリーズを企画制作している。同タイトルは2008年8月にWindows用ノベルソフトとして発売され、Yahoo!Japanトップニュースで報道された。その後2009年8月に続編が発売されている（販売：株式会社イーネット・フロンティア)。関連商品として楽曲を音楽配信とCDにより発売。本編、webラジオ等を含む関連の制作は全て自社にて行われている。 2010年10月に、初のiPad用オーディオビジュアルノベルとしてSF作家小松左京の児童向けSF「空中都市008 -アオゾラ市のものがたり-」を発売。 公式Webサイトにはライフウィズアイドルに次ぐ2011年オリジナル新作についても記載されている。 一般社団法人日本動画協会準会員（アニメーション制作会社団体）社団法人コンピュータソフトウェア著作権協会(ACCS)正会員（ソフトメーカーによる著作権保護団体)。 
* 公式Webサイト
ティルナノーグ (ゲーム会社) ; ティルナノーグ（Tirnanog Co.）は、コンピュータゲーム開発を手がける日本の会社である。本社は京都市南区に所在。
パンサーソフトウェア ; パンサーソフトウェア（Panther Software）は、株式会社パンサーソフトウェアのゲームブランド名である。恋愛シミュレーションゲームの制作が多い。旧雄図グループ。 
* 1987年：有限会社スタジオパンサー設立 
* 1991年：株式会社に改組、社名をパンサーソフトウェアに変更 
* 1999年：韓国パンサーソフトウェア、Interlex（アメリカ現地法人）を設立 
* 2000年：インターレックス株式会社に社名変更（パンサーソフトウェアの名称はブランド名

アメリカ合衆国のコンピューターゲームメーカーの情報を取得する

In [4]:
from SPARQLWrapper import SPARQLWrapper

#アメリカ合衆国のコンピューターゲームメーカーの情報を取得する
sparql3 = SPARQLWrapper(endpoint='http://ja.dbpedia.org/sparql', returnFormat='json')
sparql3.setQuery("""                                                                                                                                                                      
    PREFIX dbpedia-owl:  <http://dbpedia.org/ontology/>                                                                                                                                  
    select distinct ?name ?abstract where {
    ?company <http://dbpedia.org/ontology/wikiPageWikiLink> <http://ja.dbpedia.org/resource/Category:アメリカ合衆国のコンピュータゲームメーカー・ブランド> .
    ?company rdfs:label ?name .
    ?company <http://dbpedia.org/ontology/abstract> ?abstract .
}                                                                                                                   
""")
results3 = sparql3.query().convert()

In [5]:
#結果を整形しPrint する
for result in results3["results"]["bindings"]:
    print(result["name"]["value"],";",result["abstract"]["value"])

17-BIT ; 17-BITは、コンピューターゲームの制作会社。 ユナイテッド・ゲーム・アーティスツ、エレクトロニック・アーツなどで働いていたジェイク・カズダルが2009年にシアトルで設立した。スタジオをシアトルと京都に置く。レトロさにAIや物理演算などをプラスし、ジャンルにとらわれない作風を指向する。作品にはen:Skulls of the Shogun、en:Galak-Z: The Dimensionalがある。
2K Games ; 2K Gamesは、テイクツー・インタラクティブの子会社で、テレビゲームの販売を行う企業である。テイクツー・インタラクティブがセガから2,400万ドルでとVisual Conceptsを買収し、2005年1月25日に設立された。 2K Gamesという名前は、Visual Conceptsのゲームである2K seriesを由来としている。現在カリフォルニア州ノバトに本社を置く。
2K Marin ; 2K Marin はアメリカ合衆国カリフォルニア州ノバトとオーストラリアキャンベラに拠点を置くアメリカ合衆国とオーストラリアのコンピュータゲーム開発会社である。プレスリリースによると、スタジオの主な焦点は新規IPとほかの2K Studioとの共同開発製品の開発によせられている。 2K Marinは、ノバトスタジオBioshockのリリースの後にIrrational Games(短期間2K Boston/2K Australiaという名前であった)から派生した。2K Australia、つまりキャンベラスタジオはもともとIrrational Gamesの一部だった。新たなのゲームの発表に際して、2010年に2K Marinの一部となった。しかし、2K Marinの業績悪化のため、2K Australiaは、2011年に再び独立企業となった。
2K Sports ; 2K Sportsとは、株式会社テイクツー・インタラクティブソフトウェアの子会社である。この企業は一つの開発スタジオ、2005年にセガから買収したVisual Conceptsからなりたっている。2K SportsはおもにNBA 2Kシリーズのようなアメリカ合衆国のスポーツのコンピュータゲームを開発、発売している。
343 Industries ; 343 Industr